# Math Inverse Kinematics

In [4]:
# Import essential modules
import numpy as np
from forward_kinematics import forward_kinematics

## Create task what we'll solve

In [79]:
fT1=-74
fT2=31
fT3=19
fT4=71

In [80]:
def cos(t):
    return np.cos(np.deg2rad(t))
def sin(t):
    return np.sin(np.deg2rad(t))

In [81]:
fk = forward_kinematics(fT1,fT2,fT3,fT4)[:3,3]
fk

array([ 75., 261., 151.])

In [82]:
dx=fk[0]
dy=fk[1]
dz=fk[2]

In [83]:
dx,dy,dz

(75.0, 261.0, 151.0)

## Use math to solve IK

### Theta 1

In [84]:
T1= np.arctan(dy/dx)
T1=np.round(np.degrees(-T1))
T1

-74.0

### Theta 3

In [85]:
l1=104.
l2=89.
l3=175.
a1=95.
fi=fT2+fT3+fT4
eT2=(90-fT2)
eT3=(90-fT3)+eT2
eT4=(90-fT4)+eT3
#dz=dz-95

In [86]:
eT2,eT3,eT4

(59, 130, 149)

In [87]:
# Calculate r
r = np.sqrt(dx**2+dy**2)
r

271.56214758320056

In [88]:
# Calculate r1 if we know fT2
r1 = cos(eT2)*l1
r1

53.56395979064563

In [89]:
# Calcualte r2
r2 = r-r1
r2

217.99818779255492

In [90]:
# Find t
t = sin(eT2)*l1
t

89.14539927301968

In [91]:
t1=t-(dz-95)
t1

33.14539927301968

In [92]:
alpha1=np.arctan(r2/t1)
alpha1=np.degrees(alpha1)
alpha1

81.35471207952756

In [93]:
b=np.sqrt(t1**2+r2**2)
b

220.5035767823414

In [94]:
operation1=(l2**2+b**2-l3**2)/(2*l2*b)
operation1*1000

660.3329246975518

In [95]:
alpha2=np.arccos((l2**2+b**2-l3**2)/(2*l2*b))
alpha2=np.degrees(alpha2)
alpha2

48.67473153556754

In [96]:
T3=180-(alpha1+alpha2+fT2)
T3=np.round(T3)
T3

19.0

In [97]:
if(t1>0):
    T3=180-(alpha1+alpha2+fT2)
    T3=np.round(T3)
    print(T3)
else:
    T3=alpha1+alpha2+fT2
    T3=abs(np.round(T3))
    print(T3)

19.0


### Theta 4

In [98]:
T4=np.arccos((l2**2+l3**2-b**2)/(2*l2*l3))
T4=np.degrees(T4)
T4

108.87239466557408

In [99]:
T4=180-T4
T4=np.round(T4)
T4

71.0

In [100]:
print(f"Pred T1:{T1}, T2:{fT2}, T3:{T3} ,T4:{T4}")
print(f"True: T1:{fT1}, T2:{fT2}, T3:{fT3} ,T4:{fT4}")
print(f"Jesu li isti: {[T1,fT2,T3,T4]==[fT1,fT2,fT3,fT4]}")

Pred T1:-74.0, T2:31, T3:19.0 ,T4:71.0
True: T1:-74, T2:31, T3:19 ,T4:71
Jesu li isti: True


## Make functons for IK

In [836]:
def finder(fk, i):
    dx,dy,dz=fk
    eT2=(90-i)
    l1=104.
    l2=89.
    l3=175.
    r = np.sqrt(dx**2+dy**2)
    r1 = cos(eT2)*l1
    r2 = r-r1
    t = sin(eT2)*l1
    t1=t-(dz-95)
    if(t1!=0):        
        alpha1=np.arctan(r2/t1)
        alpha1=np.degrees(alpha1)
    else:
        alpha1=0
    b=np.sqrt(t1**2+r2**2)
    operation1=(l2**2+b**2-l3**2)/(2*l2*b)
    operation2=(l2**2+l3**2-b**2)/(2*l2*l3)
    if(int(operation1*1000)>1000 or operation1<0):
        raise Exception() 
    if(int(operation1*1000)==1000):
        operation1=np.round(operation1)
        operation2=np.round(operation2)

    alpha2=np.arccos(operation1)
    alpha2=np.degrees(alpha2)
    if(t1>0):
        T3=180-(alpha1+alpha2+i)
        T3=np.round(T3)
    else:
        T3=alpha1+alpha2+i
        T3=abs(np.round(T3))
    T4=np.arccos(operation2)
    T4=np.degrees(T4)
    T4=180-T4
    T4=np.round(T4)

    return T3, T4

def find_T2(T1,fk_true):

    mae={}
    for i in range(91):
        try:
            T3, T4 = finder(fk_true, i)
            fk =forward_kinematics(T1,i,T3,T4)[:3,3]

            if (np.array_equal(fk_true,fk)):
                return i, T3, T4

        except:
            pass
                
def calculate_inverse_kineamtics(fk):
    dx,dy,dz=fk
    # Find T1
    if(dx==0.):
        T1=90
    else:
        T1= np.arctan(dy/dx)
        T1=np.degrees(-T1)
        T1=np.round(T1)
    #Find T3
    T2, T3, T4=find_T2(T1,fk)
    return T1, T2, T3, T4

In [837]:
kT1=-74
kT2=31
kT3=19
kT4=71
fk=forward_kinematics(kT1,kT2,kT3,kT4)[:3,3]
print(fk, [kT1,kT2,kT3,kT4])

[ 75. 261. 151.] [-74, 31, 19, 71]


In [838]:
result=calculate_inverse_kineamtics(fk)
result

(-74.0, 8, 81.0, 17.0)

In [839]:
forward_kinematics(result[0],result[1],result[2],result[3])[:3,3], fk

(array([ 75., 261., 151.]), array([ 75., 261., 151.]))

In [840]:
y_true=[]
y_pred=[]
for i in range(5000):
    try:
        T1=np.random.randint(-90,91)
        T2=np.random.randint(0,91)
        T3=np.random.randint(0,91)
        T4=np.random.randint(0,91)       
        fk=forward_kinematics(T1,T2,T3,T4)[:3,3]
        result=calculate_inverse_kineamtics(fk)
        y_true.append(fk)
        y_pred.append(forward_kinematics(result[0],result[1],result[2],result[3])[:3,3])
    except:
        pass

In [841]:

from sklearn.metrics import mean_absolute_error 
mean_absolute_error(y_true,y_pred)

0.0

In [843]:
T1=np.random.randint(-90,91)
T2=np.random.randint(0,91)
T3=np.random.randint(0,91)
T4=np.random.randint(0,91)       
fk=forward_kinematics(T1,T2,T3,T4)[:3,3]
result=calculate_inverse_kineamtics(fk)
print(f"Pred T1:{result[0]}, T2:{result[1]}, T3:{result[2]} ,T4:{result[3]}")
print(f"True: T1:{T1}, T2:{T2}, T3:{T3} ,T4:{T4}")
fk_pred=forward_kinematics(result[0],result[1],result[2],result[3])[:3,3]
print(f"Jesu li isti: {np.array_equal(fk_pred,fk)}")
fk_pred, fk


Pred T1:-54.0, T2:73, T3:42.0 ,T4:28.0
True: T1:-54, T2:69, T3:64 ,T4:2
Jesu li isti: True


(array([168., 231., -52.]), array([168., 231., -52.]))

In [844]:
k=0
for i in range(50000):
    try:
        T1=np.random.randint(-90,91)
        T2=np.random.randint(0,91)
        T3=np.random.randint(0,91)
        T4=np.random.randint(0,91)       
        fk=forward_kinematics(T1,T2,T3,T4)[:3,3]
        result=calculate_inverse_kineamtics(fk)
        fk_pred=forward_kinematics(result[0],result[1],result[2],result[3])[:3,3]
        if(np.array_equal(fk_pred,fk)==False):
            #print("false")
            k+=1
    except:
        pass
print(k)

0
